# LOG Per Label 

In [1]:
    
# Import Libraries
import pandas as pd
import numpy as np
import re # Untuk text cleaning
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer # Untuk multi-label encoding
from sklearn.multioutput import MultiOutputClassifier # Untuk Binary Relevance strategy
from sklearn.linear_model import LogisticRegression # Model klasifikasi biner
from sklearn.metrics import f1_score # Metrik evaluasi
import joblib # Untuk menyimpan model dan vectorizer

# --- Tambahkan impor dan kode ini di sini ---
import nltk
from nltk.corpus import stopwords

# --- Konfigurasi ---
RANDOM_STATE = 42
TEST_SIZE = 0.2 # Ukuran validation set
MAX_FEATURES = 10000 # Batasan jumlah fitur TF-IDF
NGRAM_RANGE = (1, 2) # Menggunakan unigram dan bigram
STOP_WORDS = set(stopwords.words('english'))

# --- Langkah 1: Data Loading dan Initial Explorasi ---
print("Langkah 1: Loading Data dan Eksplorasi Awal...")
train_df = pd.read_csv('Train.csv')
test_df = pd.read_csv('Test.csv')

print("Data Training Shape:", train_df.shape)
print("Data Test Shape:", test_df.shape)

print("\nInfo Data Training:")
train_df.info()

print("\nMissing Values di Data Training:")
print(train_df.isnull().sum())

print("\nMissing Values di Data Test:")
print(test_df.isnull().sum())

# Analisis Target Variabel (labels)
# Mendapatkan daftar semua unique labels yang mungkin
all_labels = sorted(list(set([lbl for labels in train_df['labels'] for lbl in labels.split()])))
print(f"\nSemua Label SDG yang Teridentifikasi ({len(all_labels)}):")
print(all_labels)

# Hitung frekuensi setiap label (untuk memahami imbalance)
label_counts = {}
for labels in train_df['labels']:
    for label in labels.split():
        label_counts[label] = label_counts.get(label, 0) + 1

print("\nFrekuensi Setiap Label SDG:")
for label, count in sorted(label_counts.items()):
    print(f"{label}: {count}")

# Catatan: Macro F1 sangat dipengaruhi oleh performa pada label minor.
# Frekuensi rendah menunjukkan label tersebut akan sulit diprediksi.


Langkah 1: Loading Data dan Eksplorasi Awal...
Data Training Shape: (3313, 4)
Data Test Shape: (1787, 3)

Info Data Training:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3313 entries, 0 to 3312
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3313 non-null   int64 
 1   title     3313 non-null   object
 2   abstract  3313 non-null   object
 3   labels    3313 non-null   object
dtypes: int64(1), object(3)
memory usage: 103.7+ KB

Missing Values di Data Training:
id          0
title       0
abstract    0
labels      0
dtype: int64

Missing Values di Data Test:
id          0
title       0
abstract    0
dtype: int64

Semua Label SDG yang Teridentifikasi (16):
['sdg_1', 'sdg_10', 'sdg_11', 'sdg_12', 'sdg_13', 'sdg_14', 'sdg_15', 'sdg_16', 'sdg_2', 'sdg_3', 'sdg_4', 'sdg_5', 'sdg_6', 'sdg_7', 'sdg_8', 'sdg_9']

Frekuensi Setiap Label SDG:
sdg_1: 105
sdg_10: 170
sdg_11: 74
sdg_12: 187
sdg_13: 77
sdg_14: 208
sdg_15

In [2]:
# --- Langkah 2: Preprocessing Data ---
print("\nLangkah 2: Preprocessing Data...")

# Gabungkan title dan abstract
train_df['text'] = train_df['title'].fillna('') + ' ' + train_df['abstract'].fillna('')
test_df['text'] = test_df['title'].fillna('') + ' ' + test_df['abstract'].fillna('')

# Fungsi Text Cleaning
def clean_text(text): # Gunakan nama fungsi yang sudah ada di notebook Anda
    text = str(text).lower() # Pastikan input adalah string
    text = re.sub(r'[^a-z\s]', '', text) # Hapus non-alfabet dan non-spasi
    text = re.sub(r'\s+', ' ', text).strip() # Hapus spasi ganda dan trim

    # --- Tambahkan baris ini untuk menghapus stop words ---
    words = text.split()
    words = [word for word in words if word not in STOP_WORDS]
    text = " ".join(words)
    # --- Akhir penambahan ---

    return text

print("Menerapkan Text Cleaning...")
train_df['cleaned_text'] = train_df['text'].apply(clean_text)
test_df['cleaned_text'] = test_df['text'].apply(clean_text)

# Processing Label (Multi-label Binarization)
print("Menerapkan Multi-label Binarization...")
mlb = MultiLabelBinarizer()
# Fit Binarizer pada semua label yang ada di data train
y_train_bin = mlb.fit_transform(train_df['labels'].apply(lambda x: x.split()))

# Label order setelah binarisasi
print("Urutan Label setelah Binarization:", list(mlb.classes_))


Langkah 2: Preprocessing Data...
Menerapkan Text Cleaning...
Menerapkan Multi-label Binarization...
Urutan Label setelah Binarization: ['sdg_1', 'sdg_10', 'sdg_11', 'sdg_12', 'sdg_13', 'sdg_14', 'sdg_15', 'sdg_16', 'sdg_2', 'sdg_3', 'sdg_4', 'sdg_5', 'sdg_6', 'sdg_7', 'sdg_8', 'sdg_9']


In [3]:
# --- Langkah 3: Feature Engineering (TF-IDF Vectorization) ---
print("\nLangkah 3: Feature Engineering (TF-IDF)...")

# Inisialisasi TF-IDF Vectorizer
# Fit vectorizer hanya pada data training yang sudah bersih
tfidf_vectorizer = TfidfVectorizer(
    max_features=15000, # Coba tingkatkan jumlah fitur
    ngram_range=(1, 3), # Gunakan Unigram, Bigram, dan Trigram
    min_df=3, # Coba turunkan minimal dokumen yang mengandung term
)
X_train_tfidf = tfidf_vectorizer.fit_transform(train_df['cleaned_text'])
X_test_tfidf = tfidf_vectorizer.transform(test_df['cleaned_text']) # Transform data test menggunakan vectorizer yang sudah di-fit

print("Shape Data Training setelah TF-IDF:", X_train_tfidf.shape)
print("Shape Data Test setelah TF-IDF:", X_test_tfidf.shape)


Langkah 3: Feature Engineering (TF-IDF)...
Shape Data Training setelah TF-IDF: (3313, 15000)
Shape Data Test setelah TF-IDF: (1787, 15000)


In [4]:
# contoh output
import pandas as pd

# Konversi sparse matrix ke dense matrix lalu ke DataFrame
X_train_df = pd.DataFrame(
    X_train_tfidf.toarray(), 
    columns=tfidf_vectorizer.get_feature_names_out()
)

# Tampilkan beberapa baris pertama
print(X_train_df.head())


    aa  aas   ab  abdomen  abdominal  abdominal ct  abdominal pain  \
0  0.0  0.0  0.0      0.0        0.0           0.0             0.0   
1  0.0  0.0  0.0      0.0        0.0           0.0             0.0   
2  0.0  0.0  0.0      0.0        0.0           0.0             0.0   
3  0.0  0.0  0.0      0.0        0.0           0.0             0.0   
4  0.0  0.0  0.0      0.0        0.0           0.0             0.0   

   abelmoschus  abelmoschus esculentus  abi  ...   zo  zone  zones  zoom  \
0          0.0                     0.0  0.0  ...  0.0   0.0    0.0   0.0   
1          0.0                     0.0  0.0  ...  0.0   0.0    0.0   0.0   
2          0.0                     0.0  0.0  ...  0.0   0.0    0.0   0.0   
3          0.0                     0.0  0.0  ...  0.0   0.0    0.0   0.0   
4          0.0                     0.0  0.0  ...  0.0   0.0    0.0   0.0   

   zoonotic  zoothamnium   zs  zs rights  zs rights reserved  zscore  
0       0.0          0.0  0.0        0.0           

In [5]:
# --- Langkah 4 & 5: Pemilihan & Pelatihan Model, Validasi & Tuning ---
print("\nLangkah 4 & 5: Pelatihan Model, Validasi & Tuning Threshold...")

# Strategy: Binary Relevance (satu model biner per label)
# Model dasar: Logistic Regression
# Gunakan solver 'liblinear' yang baik untuk data sparse
base_model = LogisticRegression(solver='liblinear', random_state=RANDOM_STATE)
multioutput_classifier = MultiOutputClassifier(base_model, n_jobs=-1) # n_jobs=-1: gunakan semua core CPU

# Split data training untuk validasi
X_train_part, X_val, y_train_part, y_val = train_test_split(
    X_train_tfidf, y_train_bin, test_size=TEST_SIZE, random_state=RANDOM_STATE
    # Untuk stratifikasi multi-label, butuh library tambahan atau implementasi custom.
    # Untuk demo ini, kita gunakan split standar dulu.
)

print(f"Shape Data Training Part ({1-TEST_SIZE*100}%):", X_train_part.shape)
print(f"Shape Data Validation ({TEST_SIZE*100}%):", X_val.shape)

print("Memulai Pelatihan Model Binary Relevance...")
multioutput_classifier.fit(X_train_part, y_train_part)
print("Pelatihan Selesai.")

# Evaluasi pada Validation Set (Menggunakan Macro F1)
# Model Logistic Regression dengan MultiOutputClassifier akan memberikan probabilitas
y_val_proba = np.transpose([model.predict_proba(X_val)[:, 1] for model in multioutput_classifier.estimators_])


# Tuning Threshold
# Cari threshold terbaik untuk memaksimalkan Macro F1 di validation set
best_threshold = 0.5 # Default
best_macro_f1 = 0

print("Mulai Tuning Threshold...")
# Coba range threshold dari 0.1 hingga 0.9 dengan langkah 0.05
thresholds = np.arange(0.01, 1.0, 0.001)
for threshold in thresholds:
    # Konversi probabilitas menjadi prediksi biner menggunakan threshold saat ini
    y_val_pred_tuned = (y_val_proba > threshold).astype(int)

    # Hitung Macro F1 untuk threshold ini
    macro_f1 = f1_score(y_val, y_val_pred_tuned, average='macro')

    # Update threshold terbaik jika Macro F1 lebih baik
    if macro_f1 > best_macro_f1:
        best_macro_f1 = macro_f1
        best_threshold = threshold

print(f"Threshold Optimal yang ditemukan di Validation: {best_threshold:.4f}")
print(f"Macro F1 di Validation dengan Threshold Optimal: {best_macro_f1:.4f}")

# Dengan threshold optimal, buat prediksi biner untuk validation set
y_val_pred_optimal = (y_val_proba > best_threshold).astype(int)
print(f"Macro F1 di Validation (konfirmasi): {f1_score(y_val, y_val_pred_optimal, average='macro'):.4f}")




Langkah 4 & 5: Pelatihan Model, Validasi & Tuning Threshold...
Shape Data Training Part (-19.0%): (2650, 15000)
Shape Data Validation (20.0%): (663, 15000)
Memulai Pelatihan Model Binary Relevance...
Pelatihan Selesai.
Mulai Tuning Threshold...
Threshold Optimal yang ditemukan di Validation: 0.0890
Macro F1 di Validation dengan Threshold Optimal: 0.5579
Macro F1 di Validation (konfirmasi): 0.5579


In [6]:
#---------------------------------BARUUUU------------------------------------
# Cell 5: Per-Label Threshold Tuning

print("Cell 5: Per-Label Threshold Tuning...")

# Gunakan probabilitas prediksi dari model LGBM di validation set (dari Cell 4)
# Variabel y_val_proba dan y_val seharusnya sudah tersedia dari Cell 4
# Variabel thresholds (range threshold) juga harus sudah tersedia dari Cell 4
# Variabel mlb (MultiLabelBinarizer) juga harus sudah tersedia dari Cell 2
# Variabel label_classes (urutan nama label) juga harus sudah tersedia dari Cell 2

current_y_val_proba = y_val_proba # Probabilitas prediksi di validation set dari model di Cell 4
current_y_val = y_val # Ground truth biner validation set dari Cell 4

# Simpan threshold optimal untuk setiap label
best_thresholds_per_label = []
macro_f1_per_label_individual_list = [] # Opsional, simpan F1 individual

print(f"Mulai Per-Label Threshold Tuning (Mencoba {len(thresholds)} threshold per label)...")
# Pastikan label_classes (urutan nama label) sudah tersedia dari Cell 2
label_classes = list(mlb.classes_) # Ambil lagi untuk memastikan

# Loop melalui setiap label (kolom di y_val_proba)
for label_idx in range(current_y_val_proba.shape[1]):
    # print(f"  Tuning threshold untuk label: {label_classes[label_idx]}...") # Opsional, untuk melihat progress
    best_f1_for_label = -1 # F1 bisa 0, mulai dari -1
    optimal_threshold_for_label = 0.5 # Default awal

    # Dapatkan probabilitas dan target ground truth hanya untuk label ini
    label_proba = current_y_val_proba[:, label_idx]
    label_true = current_y_val[:, label_idx]

    # Coba berbagai threshold dari range 'thresholds' yang didefinisikan di Cell 4
    for threshold in thresholds:
        label_pred_tuned = (label_proba > threshold).astype(int)

        # Hitung F1 score untuk label ini
        # zero_division=1 penting jika label_true tidak memiliki positive sample di validation
        # (misal, label yang sangat jarang tidak muncul di split validation)
        f1_for_label = f1_score(label_true, label_pred_tuned, zero_division=1)

        # Update threshold terbaik untuk label ini
        if f1_for_label > best_f1_for_label:
            best_f1_for_label = f1_for_label
            optimal_threshold_for_label = threshold

    # Simpan threshold optimal dan F1 score terbaik untuk label ini
    best_thresholds_per_label.append(optimal_threshold_for_label)
    macro_f1_per_label_individual_list.append(best_f1_for_label)


# Setelah tuning untuk semua label, hitung Macro F1 total dengan threshold optimal per label
print("\nMenghitung Macro F1 Total dengan Threshold Optimal Per-Label di Validation...")
# Terapkan array threshold ke array probabilitas
y_val_pred_optimal_per_label = (current_y_val_proba > np.array(best_thresholds_per_label)).astype(int) # FIX: Gunakan current_y_val_proba
macro_f1_optimal_per_label = f1_score(current_y_val, y_val_pred_optimal_per_label, average='macro', zero_division=1)

print(f"\n--- Hasil Setelah Per-Label Threshold Tuning ---")
print(f"Threshold Optimal Per-Label:")
for i, thresh in enumerate(best_thresholds_per_label):
     print(f"  {label_classes[i]}: {thresh:.4f}")

print(f"\nMacro F1 di Validation dengan Threshold Optimal Per-Label: {macro_f1_optimal_per_label:.4f}")
# Bandingkan dengan hasil global threshold LGBM dari Cell 4
# Anda perlu mengingat skor global threshold dari output Cell 4, atau menyimpannya ke variabel di Cell 4
# print("Perbandingan dengan Hasil Sebelumnya (Global Threshold):", macro_f1_optimal_per_label - best_macro_f1_global)
print("----------------------------------------------------")

# Variabel best_thresholds_per_label ini SANGAT PENTING. Ini akan digunakan untuk prediksi akhir pada data test.

Cell 5: Per-Label Threshold Tuning...
Mulai Per-Label Threshold Tuning (Mencoba 990 threshold per label)...

Menghitung Macro F1 Total dengan Threshold Optimal Per-Label di Validation...

--- Hasil Setelah Per-Label Threshold Tuning ---
Threshold Optimal Per-Label:
  sdg_1: 0.0660
  sdg_10: 0.1040
  sdg_11: 0.0350
  sdg_12: 0.1500
  sdg_13: 0.0690
  sdg_14: 0.1220
  sdg_15: 0.1040
  sdg_16: 0.0890
  sdg_2: 0.0620
  sdg_3: 0.5130
  sdg_4: 0.0600
  sdg_5: 0.1000
  sdg_6: 0.1260
  sdg_7: 0.1650
  sdg_8: 0.1380
  sdg_9: 0.0930

Macro F1 di Validation dengan Threshold Optimal Per-Label: 0.6443
----------------------------------------------------


In [ ]:
# --- Langkah 6: Prediksi dan Submission ---
print("\nLangkah 6: Prediksi pada Test Data dan Pembuatan Submission File...")

# Dapatkan probabilitas prediksi pada data test penuh
y_test_proba = np.transpose([model.predict_proba(X_test_tfidf)[:, 1] for model in multioutput_classifier.estimators_])

# Terapkan threshold optimal yang ditemukan dari validation set
y_test_pred_bin = (y_test_proba > np.array(best_thresholds_per_label)).astype(int)

# Konversi prediksi biner kembali ke format string label
# Gunakan mlb.inverse_transform untuk mengubah array biner menjadi daftar label string
test_pred_labels = mlb.inverse_transform(y_test_pred_bin)

# Format untuk submission: gabungkan label per baris menjadi satu string
submission_labels = [" ".join(labels) for labels in test_pred_labels]

# Buat DataFrame Submission
submission_df = pd.DataFrame({'id': test_df['id'], 'labels': submission_labels})

# Simpan Submission File
submission_df.to_csv('submission_log.csv', index=False)

print("Submission file 'submission.csv' berhasil dibuat.")



Langkah 6: Prediksi pada Test Data dan Pembuatan Submission File...
Submission file 'submission.csv' berhasil dibuat.

--- Langkah Lanjutan ---
Pendekatan yang lebih powerful biasanya melibatkan Fine-tuning Model Transformer.
Contoh:
1. Gunakan library Hugging Face Transformers.
2. Pilih model pre-trained yang sesuai, misal 'allenai/scibert_scivocab_uncased' untuk domain ilmiah.
3. Siapkan Dataset dan DataLoader untuk multi-label classification.
4. Fine-tune model dengan layer klasifikasi multi-label (16 output + Sigmoid) di atasnya.
5. Latih model dengan Binary Cross-Entropy loss.
6. Validasi dan tuning threshold tetap KRUSIAL seperti di atas.
7. Pertimbangkan ensemble model Transformer (misal, rata-rata probabilitas dari beberapa run atau model yang berbeda).
Implementasi ini lebih kompleks dan membutuhkan GPU.


# lgbm per label


In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import re # Untuk text cleaning
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer # Untuk multi-label encoding
from sklearn.multioutput import MultiOutputClassifier # Untuk Binary Relevance strategy
from sklearn.linear_model import LogisticRegression # Model klasifikasi biner
from sklearn.metrics import f1_score # Metrik evaluasi
import joblib # Untuk menyimpan model dan vectorizer

# --- Tambahkan impor dan kode ini di sini ---
import nltk
from nltk.corpus import stopwords
import lightgbm as lgb # Akan digunakan nanti, import saja di awal


# --- Konfigurasi ---
RANDOM_STATE = 42
TEST_SIZE = 0.2 # Ukuran validation set
MAX_FEATURES = 10000 # Batasan jumlah fitur TF-IDF
NGRAM_RANGE = (1, 2) # Menggunakan unigram dan bigram
STOP_WORDS = set(stopwords.words('english'))

# --- Langkah 1: Data Loading dan Initial Explorasi ---
print("Langkah 1: Loading Data dan Eksplorasi Awal...")
train_df = pd.read_csv('Train.csv')
test_df = pd.read_csv('Test.csv')

print("Data Training Shape:", train_df.shape)
print("Data Test Shape:", test_df.shape)

print("\nInfo Data Training:")
train_df.info()

print("\nMissing Values di Data Training:")
print(train_df.isnull().sum())

print("\nMissing Values di Data Test:")
print(test_df.isnull().sum())

# Analisis Target Variabel (labels)
# Mendapatkan daftar semua unique labels yang mungkin
all_labels = sorted(list(set([lbl for labels in train_df['labels'] for lbl in labels.split()])))
print(f"\nSemua Label SDG yang Teridentifikasi ({len(all_labels)}):")
print(all_labels)

# Hitung frekuensi setiap label (untuk memahami imbalance)
label_counts = {}
for labels in train_df['labels']:
    for label in labels.split():
        label_counts[label] = label_counts.get(label, 0) + 1

print("\nFrekuensi Setiap Label SDG:")
for label, count in sorted(label_counts.items()):
    print(f"{label}: {count}")

# Catatan: Macro F1 sangat dipengaruhi oleh performa pada label minor.
# Frekuensi rendah menunjukkan label tersebut akan sulit diprediksi.

In [ ]:
# --- Langkah 2: Preprocessing Data ---
print("\nLangkah 2: Preprocessing Data...")

# Gabungkan title dan abstract
train_df['text'] = train_df['title'].fillna('') + ' ' + train_df['abstract'].fillna('')
test_df['text'] = test_df['title'].fillna('') + ' ' + test_df['abstract'].fillna('')

# Fungsi Text Cleaning
def clean_text(text): # Gunakan nama fungsi yang sudah ada di notebook Anda
    text = str(text).lower() # Pastikan input adalah string
    text = re.sub(r'[^a-z\s]', '', text) # Hapus non-alfabet dan non-spasi
    text = re.sub(r'\s+', ' ', text).strip() # Hapus spasi ganda dan trim

    # --- Tambahkan baris ini untuk menghapus stop words ---
    words = text.split()
    words = [word for word in words if word not in STOP_WORDS]
    text = " ".join(words)
    # --- Akhir penambahan ---

    return text

print("Menerapkan Text Cleaning...")
train_df['cleaned_text'] = train_df['text'].apply(clean_text)
test_df['cleaned_text'] = test_df['text'].apply(clean_text)

# Processing Label (Multi-label Binarization)
print("Menerapkan Multi-label Binarization...")
mlb = MultiLabelBinarizer()
# Fit Binarizer pada semua label yang ada di data train
y_train_bin = mlb.fit_transform(train_df['labels'].apply(lambda x: x.split()))

# --- Tambahkan baris ini di sini: ---Baru-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
label_classes = list(mlb.classes_) # Menyimpan urutan nama label

# Label order setelah binarisasi
print("Urutan Label setelah Binarization:", list(mlb.classes_))

In [ ]:
# --- Langkah 3: Feature Engineering (TF-IDF Vectorization) ---
print("\nLangkah 3: Feature Engineering (TF-IDF)...")

# Inisialisasi TF-IDF Vectorizer
# Fit vectorizer hanya pada data training yang sudah bersih
tfidf_vectorizer = TfidfVectorizer(
    max_features=15000, # Coba tingkatkan jumlah fitur
    ngram_range=(1, 3), # Gunakan Unigram, Bigram, dan Trigram
    min_df=3 # Coba turunkan minimal dokumen yang mengandung term
    # Optional: coba parameter lain seperti sublinear_tf=True, smooth_idf=True
    # sublinear_tf=True,
    # smooth_idf=True
) # min_df=5: abaikan term yang muncul di kurang dari 5 dokumen
X_train_tfidf = tfidf_vectorizer.fit_transform(train_df['cleaned_text'])
X_test_tfidf = tfidf_vectorizer.transform(test_df['cleaned_text']) # Transform data test menggunakan vectorizer yang sudah di-fit

print("Shape Data Training setelah TF-IDF:", X_train_tfidf.shape)
print("Shape Data Test setelah TF-IDF:", X_test_tfidf.shape)

In [ ]:
# --- Langkah 4 & 5: Pemilihan & Pelatihan Model, Validasi & Tuning ---
print("\nLangkah 4 & 5: Pelatihan Model, Validasi & Tuning Threshold...")

# Strategy: Binary Relevance (satu model biner per label)
# Model dasar: Logistic Regression
# Gunakan solver 'liblinear' yang baik untuk data sparse
base_model = lgb.LGBMClassifier(objective='binary', metric='binary_logloss', random_state=RANDOM_STATE)
multioutput_classifier = MultiOutputClassifier(base_model, n_jobs=-1) # n_jobs=-1: gunakan semua core CPU

# Split data training untuk validasi
X_train_part, X_val, y_train_part, y_val = train_test_split(
    X_train_tfidf, y_train_bin, test_size=TEST_SIZE, random_state=RANDOM_STATE
    # Untuk stratifikasi multi-label, butuh library tambahan atau implementasi custom.
    # Untuk demo ini, kita gunakan split standar dulu.
)

print(f"Shape Data Training Part ({1-TEST_SIZE*100}%):", X_train_part.shape)
print(f"Shape Data Validation ({TEST_SIZE*100}%):", X_val.shape)

print("Memulai Pelatihan Model Binary Relevance...")
multioutput_classifier.fit(X_train_part, y_train_part)
print("Pelatihan Selesai.")

# Evaluasi pada Validation Set (Menggunakan Macro F1)
# Model Logistic Regression dengan MultiOutputClassifier akan memberikan probabilitas
y_val_proba = np.transpose([model.predict_proba(X_val)[:, 1] for model in multioutput_classifier.estimators_])


# Tuning Threshold
# Cari threshold terbaik untuk memaksimalkan Macro F1 di validation set
best_threshold = 0.05 # Default
best_macro_f1 = 0

print("Mulai Tuning Threshold...")
# Coba range threshold dari 0.1 hingga 0.9 dengan langkah 0.05
thresholds = np.arange(0.01, 1.0, 0.0001) #0.0001
for threshold in thresholds:
    # Konversi probabilitas menjadi prediksi biner menggunakan threshold saat ini
    y_val_pred_tuned = (y_val_proba > threshold).astype(int)

    # Hitung Macro F1 untuk threshold ini
    macro_f1 = f1_score(y_val, y_val_pred_tuned, average='macro')

    # Update threshold terbaik jika Macro F1 lebih baik
    if macro_f1 > best_macro_f1:
        best_macro_f1 = macro_f1
        best_threshold = threshold

print(f"Threshold Optimal yang ditemukan di Validation: {best_threshold:.4f}")
print(f"Macro F1 di Validation dengan Threshold Optimal: {best_macro_f1:.4f}")

# Dengan threshold optimal, buat prediksi biner untuk validation set
y_val_pred_optimal = (y_val_proba > best_threshold).astype(int)
print(f"Macro F1 di Validation (konfirmasi): {f1_score(y_val, y_val_pred_optimal, average='macro'):.4f}")

In [ ]:
#---------------------------------BARUUUU------------------------------------
# Cell 5: Per-Label Threshold Tuning

print("Cell 5: Per-Label Threshold Tuning...")

# Gunakan probabilitas prediksi dari model LGBM di validation set (dari Cell 4)
# Variabel y_val_proba dan y_val seharusnya sudah tersedia dari Cell 4
# Variabel thresholds (range threshold) juga harus sudah tersedia dari Cell 4
# Variabel mlb (MultiLabelBinarizer) juga harus sudah tersedia dari Cell 2
# Variabel label_classes (urutan nama label) juga harus sudah tersedia dari Cell 2

current_y_val_proba = y_val_proba # Probabilitas prediksi di validation set dari model di Cell 4
current_y_val = y_val # Ground truth biner validation set dari Cell 4

# Simpan threshold optimal untuk setiap label
best_thresholds_per_label = []
macro_f1_per_label_individual_list = [] # Opsional, simpan F1 individual

print(f"Mulai Per-Label Threshold Tuning (Mencoba {len(thresholds)} threshold per label)...")
# Pastikan label_classes (urutan nama label) sudah tersedia dari Cell 2
label_classes = list(mlb.classes_) # Ambil lagi untuk memastikan

# Loop melalui setiap label (kolom di y_val_proba)
for label_idx in range(current_y_val_proba.shape[1]):
    # print(f"  Tuning threshold untuk label: {label_classes[label_idx]}...") # Opsional, untuk melihat progress
    best_f1_for_label = -1 # F1 bisa 0, mulai dari -1
    optimal_threshold_for_label = 0.5 # Default awal

    # Dapatkan probabilitas dan target ground truth hanya untuk label ini
    label_proba = current_y_val_proba[:, label_idx]
    label_true = current_y_val[:, label_idx]

    # Coba berbagai threshold dari range 'thresholds' yang didefinisikan di Cell 4
    for threshold in thresholds:
        label_pred_tuned = (label_proba > threshold).astype(int)

        # Hitung F1 score untuk label ini
        # zero_division=1 penting jika label_true tidak memiliki positive sample di validation
        # (misal, label yang sangat jarang tidak muncul di split validation)
        f1_for_label = f1_score(label_true, label_pred_tuned, zero_division=1)

        # Update threshold terbaik untuk label ini
        if f1_for_label > best_f1_for_label:
            best_f1_for_label = f1_for_label
            optimal_threshold_for_label = threshold

    # Simpan threshold optimal dan F1 score terbaik untuk label ini
    best_thresholds_per_label.append(optimal_threshold_for_label)
    macro_f1_per_label_individual_list.append(best_f1_for_label)


# Setelah tuning untuk semua label, hitung Macro F1 total dengan threshold optimal per label
print("\nMenghitung Macro F1 Total dengan Threshold Optimal Per-Label di Validation...")
# Terapkan array threshold ke array probabilitas
y_val_pred_optimal_per_label = (current_y_val_proba > np.array(best_thresholds_per_label)).astype(int) # FIX: Gunakan current_y_val_proba
macro_f1_optimal_per_label = f1_score(current_y_val, y_val_pred_optimal_per_label, average='macro', zero_division=1)

print(f"\n--- Hasil Setelah Per-Label Threshold Tuning ---")
print(f"Threshold Optimal Per-Label:")
for i, thresh in enumerate(best_thresholds_per_label):
     print(f"  {label_classes[i]}: {thresh:.4f}")

print(f"\nMacro F1 di Validation dengan Threshold Optimal Per-Label: {macro_f1_optimal_per_label:.4f}")
# Bandingkan dengan hasil global threshold LGBM dari Cell 4
# Anda perlu mengingat skor global threshold dari output Cell 4, atau menyimpannya ke variabel di Cell 4
# print("Perbandingan dengan Hasil Sebelumnya (Global Threshold):", macro_f1_optimal_per_label - best_macro_f1_global)
print("----------------------------------------------------")

# Variabel best_thresholds_per_label ini SANGAT PENTING. Ini akan digunakan untuk prediksi akhir pada data test.

In [ ]:
# --- Langkah 6: Prediksi dan Submission ---
print("\nLangkah 6: Prediksi pada Test Data dan Pembuatan Submission File...")

# Dapatkan probabilitas prediksi pada data test penuh
y_test_proba = np.transpose([model.predict_proba(X_test_tfidf)[:, 1] for model in multioutput_classifier.estimators_])

# Terapkan threshold optimal yang ditemukan dari validation set
# y_test_pred_bin = (y_test_proba > best_threshold).astype(int)
y_test_pred_bin = (y_test_proba > np.array(best_thresholds_tuned)).astype(int)

# Konversi prediksi biner kembali ke format string label
# Gunakan mlb.inverse_transform untuk mengubah array biner menjadi daftar label string
test_pred_labels = mlb.inverse_transform(y_test_pred_bin)

# Format untuk submission: gabungkan label per baris menjadi satu string
submission_labels = [" ".join(labels) for labels in test_pred_labels]

# Buat DataFrame Submission
submission_df = pd.DataFrame({'id': test_df['id'], 'labels': submission_labels})

# Simpan Submission File
submission_df.to_csv('submission_lgbm_per_label.csv', index=False)

print("Submission file 'submission.csv' berhasil dibuat.")


# scibert 1

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import re # Untuk text cleaning
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer # Untuk multi-label encoding
from sklearn.multioutput import MultiOutputClassifier # Untuk Binary Relevance strategy
from sklearn.linear_model import LogisticRegression # Model klasifikasi biner
from sklearn.metrics import f1_score # Metrik evaluasi
import joblib # Untuk menyimpan model dan vectorizer

from transformers import AutoTokenizer, AutoModel
import torch
from torch.utils.data import Dataset, DataLoader
import warnings

from transformers import get_scheduler

# --- Tambahkan impor dan kode ini di sini ---
import nltk
from nltk.corpus import stopwords
import lightgbm as lgb # Akan digunakan nanti, import saja di awal


# --- Konfigurasi ---
RANDOM_STATE = 42
TEST_SIZE = 0.2 # Ukuran validation set
MAX_FEATURES = 10000 # Batasan jumlah fitur TF-IDF
NGRAM_RANGE = (1, 2) # Menggunakan unigram dan bigram
STOP_WORDS = set(stopwords.words('english'))
# --- Tambahkan konfigurasi Transformer di sini ---
MODEL_NAME = 'allenai/scibert_scivocab_uncased' # Model pre-trained SciBERT
MAX_LEN = 256 # Panjang maksimum sequence token. Abstract bisa panjang. Sesuaikan ini.
              # Meningkatkan MAX_LEN butuh lebih banyak VRAM/GPU. 256 adalah titik awal yang wajar.
              # Pertimbangkan truncation=True saat t    okenizing jika teks lebih panjang.
BATCH_SIZE = 16 # Ukuran batch untuk DataLoader. Sesuaikan dengan VRAM GPU Anda.
                # Ukuran lebih besar bisa mempercepat pelatihan, tapi butuh lebih banyak VRAM.
EPOCHS = 4 # Jumlah epoch pelatihan. Mulai dari nilai kecil.
LEARNING_RATE = 2e-5 # Learning rate untuk fine-tuning Transformer. Nilai kecil standar (misal 1e-5, 2e-5, 3e-5).

print(f"Menggunakan model: {MODEL_NAME}")
# --- Tambahkan definisi range threshold juga di sini (jika belum ada atau mau disesuaikan) ---
# Ini range threshold untuk tuning per-label nanti
# thresholds_to_try = np.arange(0.01, 1.0, 0.0001) # Dari Cell 4 Anda
# Jika mau lebih fine-grained di area probabilitas rendah:
thresholds_to_try = np.concatenate([np.arange(0.001, 0.01, 0.001), np.arange(0.01, 1.0, 0.01)]) # Lebih banyak di range rendah
print(f"Range Thresholds untuk Tuning Per-Label: {thresholds_to_try[0]:.4f} - {thresholds_to_try[-1]:.4f} (Jumlah: {len(thresholds_to_try)})")

# --- Langkah 1: Data Loading dan Initial Explorasi ---
print("Langkah 1: Loading Data dan Eksplorasi Awal...")
train_df = pd.read_csv('Train.csv')
test_df = pd.read_csv('Test.csv')

print("Data Training Shape:", train_df.shape)
print("Data Test Shape:", test_df.shape)

print("\nInfo Data Training:")
train_df.info()

print("\nMissing Values di Data Training:")
print(train_df.isnull().sum())

print("\nMissing Values di Data Test:")
print(test_df.isnull().sum())

# Analisis Target Variabel (labels)
# Mendapatkan daftar semua unique labels yang mungkin
all_labels = sorted(list(set([lbl for labels in train_df['labels'] for lbl in labels.split()])))
print(f"\nSemua Label SDG yang Teridentifikasi ({len(all_labels)}):")
print(all_labels)

# Hitung frekuensi setiap label (untuk memahami imbalance)
label_counts = {}
for labels in train_df['labels']:
    for label in labels.split():
        label_counts[label] = label_counts.get(label, 0) + 1

print("\nFrekuensi Setiap Label SDG:")
for label, count in sorted(label_counts.items()):
    print(f"{label}: {count}")

# Catatan: Macro F1 sangat dipengaruhi oleh performa pada label minor.
# Frekuensi rendah menunjukkan label tersebut akan sulit diprediksi.

# Pastikan device (GPU/CPU) terdeteksi
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Menggunakan device:")


In [ ]:
# --- Langkah 2: Preprocessing Data ---
print("\nLangkah 2: Preprocessing Data...")

# Gabungkan title dan abstract
train_df['text'] = train_df['title'].fillna('') + ' ' + train_df['abstract'].fillna('')
test_df['text'] = test_df['title'].fillna('') + ' ' + test_df['abstract'].fillna('')

# Fungsi Text Cleaning
def clean_text(text): # Gunakan nama fungsi yang sudah ada di notebook Anda
    text = str(text).lower() # Pastikan input adalah string
    text = re.sub(r'[^a-z\s]', '', text) # Hapus non-alfabet dan non-spasi
    text = re.sub(r'\s+', ' ', text).strip() # Hapus spasi ganda dan trim

    # --- Tambahkan baris ini untuk menghapus stop words ---
    words = text.split()
    words = [word for word in words if word not in STOP_WORDS]
    text = " ".join(words)
    # --- Akhir penambahan ---

    return text

print("Menerapkan Text Cleaning...")
train_df['cleaned_text'] = train_df['text'].apply(clean_text)
test_df['cleaned_text'] = test_df['text'].apply(clean_text)

# Processing Label (Multi-label Binarization)
print("Menerapkan Multi-label Binarization...")
mlb = MultiLabelBinarizer()
# Fit Binarizer pada semua label yang ada di data train
y_train_bin = mlb.fit_transform(train_df['labels'].apply(lambda x: x.split()))

# --- Tambahkan baris ini di sini: ---Baru-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
label_classes = list(mlb.classes_) # Menyimpan urutan nama label

# --- Tambahkan kode ini di sini ------------------------------------------------------------------------
print("\nMenghitung Bobot Label untuk Weighted BCE Loss...")
# Total jumlah sampel
total_samples = len(train_df)
# Jumlah kemunculan SETIAP label secara total di SELURUH dataset
# Ini BUKAN jumlah dokumen, ini jumlah total '1' di y_train_bin
label_occurrences = np.sum(y_train_bin, axis=0)

# Hitung bobot. Rumus umum: N_total / (N_kelas * N_kelas_total). Atau N_total / N_kelas.
# Coba rumus sederhana: total_samples / label_occurrences
# Atau lebih umum: total_samples / (jumlah_label * label_occurrences)
# Atau inverse frequency: 1 / label_occurrences, lalu normalisasi
# Rumus yang umum dan sering berhasil untuk multi-label:
# Pos_weight[i] = (Total Negatif untuk Label i) / (Total Positif untuk Label i)
# Total Negatif untuk Label i = total_samples - label_occurrences[i]

label_weights = (total_samples - label_occurrences) / label_occurrences
# PENTING: Urutan bobot harus sama dengan urutan label di mlb.classes_
# label_classes = list(mlb.classes_) # mlb.classes_ sudah punya urutan yang benar
# Pastikan label_occurrences juga dalam urutan yang benar, np.sum(y_train_bin, axis=0) sudah benar urutannya.

# Konversi bobot ke tensor PyTorch dan pindahkan ke device
# Bobot harus float
label_weights_tensor = torch.tensor(label_weights, dtype=torch.float).to(device)

print("Bobot Label (dalam urutan label_classes):")
for i, label in enumerate(label_classes):
    print(f"  {label}: {label_weights[i]:.2f}")

print(f"\nBobot Label Tensor shape: {label_weights_tensor.shape}")
print(f"Bobot Label Tensor device: {label_weights_tensor.device}")
# --- Akhir penambahan -----------------------------------------------------------------------------------------------------------



# Label order setelah binarisasi
print("Urutan Label setelah Binarization:", list(mlb.classes_))

In [ ]:
# Cell 3: Transformer Data Preparation - Tokenization, Datasets, DataLoaders

print("Cell 3: Transformer Data Preparation...")

# Gunakan kolom 'cleaned_text' dari Cell 2
# Gunakan y_train_bin dari Cell 2
# Gunakan RANDOM_STATE dan TEST_SIZE dari Cell 1

# --- Load Tokenizer ---
# Tokenizer ini yang akan mengubah teks menjadi ID numerik sesuai vocabulary model.
# Ini menggantikan TfidfVectorizer
print(f"Memuat tokenizer: {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print("Tokenizer dimuat.")

# --- Tokenisasi Data ---
print(f"Menerapkan tokenisasi ke data training dan test (max_len={MAX_LEN})...")

# Fungsi untuk melakukan tokenisasi dan mengembalikan PyTorch tensors
def tokenize_and_tensorize(text):
    encoded = tokenizer.encode_plus(
        text,
        add_special_tokens=True,    # Tambahkan token [CLS] dan [SEP]
        max_length=MAX_LEN,         # Padding/Truncation ke panjang maksimum
        padding='max_length',       # Pad ke max_length
        return_attention_mask=True, # Buat attention mask
        return_tensors='pt',        # Kembalikan sebagai PyTorch tensors
        truncation=True             # Lakukan truncation jika lebih panjang dari max_len
    )
    # Squeeze untuk menghilangkan dimensi batch=1 dari output encode_plus
    return {
        'input_ids': encoded['input_ids'].squeeze(),
        'attention_mask': encoded['attention_mask'].squeeze()
    }

# Tokenisasi semua teks. Hasilnya list of dictionaries.
print("Tokenizing train text...")
train_encoded = [tokenize_and_tensorize(text) for text in train_df['cleaned_text']]
print("Tokenizing test text...")
test_encoded = [tokenize_and_tensorize(text) for text in test_df['cleaned_text']]
print("Tokenization selesai.")


# Pisahkan input_ids dan attention_mask ke dalam list terpisah
train_input_ids = torch.stack([x['input_ids'] for x in train_encoded])
train_attention_masks = torch.stack([x['attention_mask'] for x in train_encoded])

test_input_ids = torch.stack([x['input_ids'] for x in test_encoded])
test_attention_masks = torch.stack([x['attention_mask'] for x in test_encoded])

print(f"Shape train_input_ids: {train_input_ids.shape}")
print(f"Shape train_attention_masks: {train_attention_masks.shape}")
print(f"Shape test_input_ids: {test_input_ids.shape}")
print(f"Shape test_attention_masks: {test_attention_masks.shape}")

# --- Data Biner Label (Sudah ada dari Cell 2, y_train_bin) ---
# Konversi y_train_bin ke PyTorch Tensor (tipe float untuk loss)
y_train_bin_tensor = torch.tensor(y_train_bin, dtype=torch.float)
print(f"Shape y_train_bin_tensor: {y_train_bin_tensor.shape}")


# --- Split Data Training untuk Validasi (Menggunakan Tensors) ---
# Ini menggantikan split di Cell 4 sebelumnya
print(f"\nMelakukan split data training ({1-TEST_SIZE*100}% train, {TEST_SIZE*100}% val)...")
train_indices, val_indices = train_test_split(
    np.arange(len(train_df)), # Split indeksnya
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    # Multi-label stratification is complex, skip for now or use specialized lib
    # stratify=y_train_bin # Simple stratification might not work well for multi-label
)

# Ambil tensor berdasarkan indeks split
X_train_input_ids = train_input_ids[train_indices]
X_train_attention_masks = train_attention_masks[train_indices]
y_train_split = y_train_bin_tensor[train_indices]

X_val_input_ids = train_input_ids[val_indices]
X_val_attention_masks = train_attention_masks[val_indices]
y_val_split = y_train_bin_tensor[val_indices]

print(f"Shape train_part input_ids: {X_train_input_ids.shape}")
print(f"Shape val input_ids: {X_val_input_ids.shape}")


# --- Membuat PyTorch Datasets dan DataLoaders ---
print("\nMembuat PyTorch Datasets dan DataLoaders...")

# Dataset untuk train_part, validation, dan test
train_dataset = torch.utils.data.TensorDataset(X_train_input_ids, X_train_attention_masks, y_train_split)
val_dataset = torch.utils.data.TensorDataset(X_val_input_ids, X_val_attention_masks, y_val_split)
# Untuk data test, kita hanya butuh input dan attention mask, targetnya tidak ada
test_dataset = torch.utils.data.TensorDataset(test_input_ids, test_attention_masks)

# DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False) # Jangan shuffle validation
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False) # Jangan shuffle test

print("Datasets dan DataLoaders selesai dibuat.")

print("\nData Preparation untuk Transformer Selesai.")
print("Langkah selanjutnya adalah membuat arsitektur model Transformer dan training loop.")

In [ ]:
# Cell 4: Transformer Model Definition, Optimizer, Loss

print("Cell 4: Transformer Model Definition, Optimizer, Loss...")

# Gunakan MODEL_NAME dari Cell 1
# Gunakan device dari Cell 1
# Gunakan jumlah label dari y_train_bin atau label_classes (Cell 2)
NUM_LABELS = len(label_classes) # Jumlah label SDG (16)

# --- Definisikan Arsitektur Model Transformer Multi-label ---
# Kita akan menggunakan AutoModel (base Transformer tanpa head klasifikasi)
# dan menambahkan layer linear (Dense) di atasnya untuk multi-label klasifikasi
class TransformerMultiLabelClassifier(torch.nn.Module):
    def __init__(self, model_name, num_labels):
        super(TransformerMultiLabelClassifier, self).__init__()
        # Muat model Transformer pre-trained
        # Setting output_hidden_states=True bisa berguna, tapi default False sudah cukup
        self.transformer = AutoModel.from_pretrained(model_name)
        # Tambahkan layer klasifikasi linear di atas output [CLS] token
        # Ukuran input layer linear adalah dimensi embedding dari model Transformer
        # Misalnya, untuk SciBERT base uncased, hidden size-nya 768
        self.classifier = torch.nn.Linear(self.transformer.config.hidden_size, num_labels)
        # Fungsi aktivasi Sigmoid untuk output (probabilitas independen per label)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, input_ids, attention_mask):
        # Masukkan input ke model Transformer
        outputs = self.transformer(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        # Ambil output [CLS] token. Ini biasanya output pertama di sequence_output.
        # Shape: (batch_size, sequence_length, hidden_size)
        sequence_output = outputs.last_hidden_state
        cls_token_output = sequence_output[:, 0, :] # Ambil vektor untuk token [CLS] (indeks 0)

        # Masukkan output [CLS] ke layer klasifikasi
        logits = self.classifier(cls_token_output)
        # Apply Sigmoid untuk mendapatkan probabilitas
        probabilities = self.sigmoid(logits)

        return probabilities, logits # Kembalikan probabilitas (untuk prediksi) dan logits (untuk loss)

# --- Inisialisasi Model dan Pindah ke Device (GPU/CPU) ---
print("Menginisialisasi model Transformer...")
model = TransformerMultiLabelClassifier(MODEL_NAME, NUM_LABELS)
model.to(device) # Pindahkan model ke GPU (atau CPU)
print("Model diinisialisasi.")

# --- Definisikan Optimizer dan Loss Function ---
# Optimizer: AdamW seringkali standar untuk fine-tuning Transformer
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

# Loss Function: Binary Cross-Entropy with Logits (BCELoss) untuk multi-label classification
#--------------------------------------------------------------------------------------------------------------------------------------------------------baru
criterion = torch.nn.BCEWithLogitsLoss(pos_weight=label_weights_tensor) # Gunakan bobot label
print("Optimizer dan Weighted BCEWithLogitsLoss Function diinisialisasi.")

print("Optimizer dan Loss Function diinisialisasi.")

print("\nModel Setup Selesai.")
print("Langkah selanjutnya adalah menulis training loop.")



In [ ]:
# Cell 5: Transformer Training Loop

print("Cell 5: Transformer Training Loop...")

# Gunakan model, optimizer, criterion, device dari Cell 4
# Gunakan train_dataloader dan val_dataloader dari Cell 3
# Gunakan EPOCHS dari Cell 1

# --- Training Function (Optional, tapi baik untuk modularitas) ---
def train_epoch(model, dataloader, optimizer, criterion, device):
    model.train() # Set model ke mode training
    total_loss = 0
    # Loop melalui batch data training
    for batch in dataloader:
        # Pindahkan batch data ke device (GPU/CPU)
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device) # Target biner label

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        # Kita butuh logits untuk BCELoss
        probabilities, logits = model(input_ids=input_ids, attention_mask=attention_mask)

        # Hitung loss
        loss = criterion(logits, labels)

        # Backward pass dan optimasi
        loss.backward()
        optimizer.step()

        total_loss += loss.item() # Kumpulkan loss per batch

    return total_loss / len(dataloader) # Rata-rata loss per epoch


# --- Evaluation Function (Optional) ---
def evaluate_epoch(model, dataloader, criterion, device):
    model.eval() # Set model ke mode evaluation
    total_loss = 0
    all_probabilities = []
    all_labels = []

    # Nonaktifkan perhitungan gradient saat evaluasi
    with torch.no_grad():
        # Loop melalui batch data validation
        for batch in dataloader:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels = batch[2].to(device)

            # Forward pass
            probabilities, logits = model(input_ids=input_ids, attention_mask=attention_mask)

            # Hitung loss (optional, untuk monitor)
            loss = criterion(logits, labels)
            total_loss += loss.item()

            # Kumpulkan probabilitas dan ground truth untuk evaluasi F1 nanti
            all_probabilities.append(probabilities.cpu().numpy()) # Pindahkan ke CPU untuk numpy
            all_labels.append(labels.cpu().numpy()) # Pindahkan ke CPU untuk numpy

    avg_loss = total_loss / len(dataloader)
    all_probabilities = np.concatenate(all_probabilities)
    all_labels = np.concatenate(all_labels)

    # Kembalikan probabilitas dan label untuk threshold tuning
    return avg_loss, all_probabilities, all_labels


# --- Main Training Loop ---
print(f"\nMemulai Training Loop ({EPOCHS} epochs)...")

total_steps = len(train_dataloader) * EPOCHS
scheduler = get_scheduler(
    "linear", # Jenis scheduler
    optimizer=optimizer,
    num_warmup_steps=0, # Mulai tanpa warmup (bisa diatur > 0)
    num_training_steps=total_steps
)

print(f"\nTotal langkah pelatihan: {total_steps}")
print("Learning Rate Scheduler diinisialisasi.")


best_val_macro_f1 = -1 # Lacak Macro F1 terbaik di validation
best_val_thresholds = None # Simpan thresholds terbaik

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")

    # Latih
    train_loss = train_epoch(model, train_dataloader, optimizer, criterion, device)
    print(f"  Training Loss: {train_loss:.4f}")

    # Evaluasi
    val_loss, val_probabilities, val_labels = evaluate_epoch(model, val_dataloader, criterion, device)
    print(f"  Validation Loss: {val_loss:.4f}")

    # --- Lakukan Per-Label Threshold Tuning di Validation Set setelah setiap epoch ---
    # Ini seperti kode dari Cell 5 sebelumnya, tapi menggunakan probabilitas dari model Transformer
    current_y_val_proba = val_probabilities # Probabilitas dari evaluate_epoch
    current_y_val = val_labels # Ground truth dari evaluate_epoch

    best_thresholds_current_epoch = []
    macro_f1_per_label_individual_current_epoch = []

    # Gunakan thresholds_to_try dari Cell 1
    # Gunakan label_classes dari Cell 2

    for label_idx in range(current_y_val_proba.shape[1]):
        best_f1_for_label = -1
        optimal_threshold_for_label = 0.5

        label_proba = current_y_val_proba[:, label_idx]
        label_true = current_y_val[:, label_idx]

        for threshold in thresholds_to_try: # Gunakan range threshold yang didefinisikan di Cell 1
            label_pred_tuned = (label_proba > threshold).astype(int)
            f1_for_label = f1_score(label_true, label_pred_tuned, zero_division=1)

            if f1_for_label > best_f1_for_label:
                best_f1_for_label = f1_for_label
                optimal_threshold_for_label = threshold

        best_thresholds_current_epoch.append(optimal_threshold_for_label)
        macro_f1_per_label_individual_current_epoch.append(best_f1_for_label)


    # Hitung Macro F1 Total untuk epoch ini
    y_val_pred_optimal_current_epoch = (current_y_val_proba > np.array(best_thresholds_current_epoch)).astype(int)
    macro_f1_current_epoch = f1_score(current_y_val, y_val_pred_optimal_current_epoch, average='macro', zero_division=1)

    print(f"  Validation Macro F1 (dengan Threshold Optimal Per-Label): {macro_f1_current_epoch:.4f}")

    # Simpan model dan thresholds jika ini adalah epoch terbaik
    if macro_f1_current_epoch > best_val_macro_f1:
        best_val_macro_f1 = macro_f1_current_epoch
        best_val_thresholds = best_thresholds_current_epoch.copy() # Simpan copy-nya
        print("  ---> Macro F1 di Validation Meningkat. Menyimpan thresholds terbaik.")
        # Anda bisa tambahkan menyimpan model state_dict di sini jika mau
        # torch.save(model.state_dict(), 'best_transformer_model.pth')

print("\nTraining Loop Selesai.")
print(f"Macro F1 terbaik di Validation: {best_val_macro_f1:.4f}")

In [ ]:
# Cell 6: Transformer Prediction on Test and Submission

print("Cell 6: Transformer Prediction and Submission...")

# Gunakan model (yang sudah dilatih di Cell 5)
# Gunakan test_dataloader dari Cell 3
# Gunakan device dari Cell 1
# Gunakan best_val_thresholds dari Cell 5
# Gunakan mlb dari Cell 2
# Gunakan test_df['id'] dari Cell 1

# --- Prediksi Probabilitas pada Test Data ---
print("Membuat prediksi probabilitas pada data test menggunakan model Transformer...")

model.eval() # Set model ke mode evaluation
test_probabilities = []

with torch.no_grad(): # Nonaktifkan perhitungan gradient
    for batch in test_dataloader:
        # Data test tidak punya label, hanya input_ids dan attention_mask
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)

        # Forward pass
        probabilities, logits = model(input_ids=input_ids, attention_mask=attention_mask)

        # Kumpulkan probabilitas
        test_probabilities.append(probabilities.cpu().numpy())

# Gabungkan semua probabilitas dari batch
test_probabilities = np.concatenate(test_probabilities)

print("Prediksi probabilitas pada data test selesai.")
print(f"Shape test_probabilities: {test_probabilities.shape}")


# --- Terapkan Threshold Optimal Per-Label ---
# Gunakan best_val_thresholds yang ditemukan di Cell 5
# Pastikan best_val_thresholds tersedia di memori

print("\nMenerapkan threshold optimal per-label untuk konversi biner...")
# best_val_thresholds adalah list, konversi ke numpy array untuk perbandingan
test_pred_bin_final = (test_probabilities > np.array(best_val_thresholds)).astype(int)
print("Penerapan threshold selesai.")


# --- Konversi Prediksi Biner ke Format String Label ---
# Gunakan mlb (MultiLabelBinarizer) dari Cell 2
print("\nMengkonversi prediksi biner ke string label...")
test_pred_labels_final = mlb.inverse_transform(test_pred_bin_final) # mlb dari Cell 2

# Format untuk submission
submission_labels_final = [" ".join(labels) for labels in test_pred_labels_final]
print("Konversi ke string label selesai.")


# --- Buat DataFrame Submission dan Simpan File ---
print("\nMembuat DataFrame submission...")
submission_df_final = pd.DataFrame({'id': test_df['id'], 'labels': submission_labels_final}) # test_df dari Cell 1

# Simpan Submission File
print("\nMenyimpan submission file 'submission.csv'...")
submission_df_final.to_csv('submission_scibert.csv', index=False)

print("\n--- Submission file 'submission.csv' berhasil dibuat ---")

# join

In [ ]:
import pandas as pd

# Load kedua fn_filled = pd.ren_filled.csv')
submission_log = pd.read_csv('submission_lgbm_per_label.csv')


# Cek dulu kolom kuncinya, misalnya pakai 'id' atau 'filename'
# Misal kita pakai 'id' sebagai kunci
key_column = 'id'  # ganti sesuai kolom unik yang dipakai di kedua file

# Merge dua DataFrame berdasarkan key
mn_filled.merge(submission_log[[key_column, 'labels']], on=key_column, how='left', suffixes=('', '_log'))

# Isi nilai yang kosong di 'labels' dengan 'labels_log' dari submission_lgbm.csv
merged_final['labels'] = merged['labels'].fillna(merged['labels_log'])

# Hapus kolom bantu
merged = merged.drop(columns=['labels_log'])

# Simpan hasilnya
merged.to_csv('submission_filled.csv', index=False)

print("Done! Hasil disimpan di submission_filled.csv")

# Merged 2
submission_filled = pd.read_csv("submission_filled.csv")
submission_log = pd.read_csv('submission_log.csv')

key_column = 'id'  # ganti sesuai kolom unik yang dipakai di kedua file

# Merge dua DataFrame berdasarkan key
merged_final = submission_filled.merge(submission_log[[key_column, 'labels']], on=key_column, how='left', suffixes=('', '_log'))

# Isi nilai yang kosong di 'labels' dengan 'labels_log' dari submission_log.csv
merged_final['labels'] = merged_final['labels'].fillna(merged_final['labels_log'])

# Hapus kolom bantu
merged_final = merged_final.drop(columns=['labels_log'])

# Simpan hasilnya
merged_final.to_csv('submission_final.csv', index=False)

print("Done! Hasil disimpan di submission_filled.csv")